<div class="alert alert-block alert-info text-center">
    <H1> Quick Introduction </H1>
</div>

### THIS COURSE WILL SHOW YOU HOW TO:

Impress your manager in the next KPI's meeting!
- Impress your boss or colleagues with a fancy **interactive dashboard** that beats any spreadsheet. It might sound silly if you work in a full-fledged data-driven company with plenty of data scientists, but it is definitely true if you work in a place where Excel is king - i.e. almost everywhere!
<br><br>

Adapt the code to your projects
- **Adapt this code to any of your projects** and explore new ways to tell your own story. Even though we are building a portfolio tracker in the course, **you do not need to have any interest in the financial markets**! You can replace the charts with the ones from your own projects and the code will still work.
<br><br>

Create a cool interactive dashboard from scratch
- Create a dashboard from scratch by using **Plotly** for the charts, **Dash** for the app, and **Bootstrap** to style it up. I will make sure you understand the basics first with a few examples and quizzes, and then we use them to develop the whole dashboard as we go. You will see the **exact process I use to create dashboards**, but **you are free to adapt it**!
<br><br>

Freelance your dashboarding skills
- Believe it or not, there is a **high demand** for people who can create dashboards and put together engaging stories with data. If you are looking for a sidegig there are plenty of customers on Fiverr and Upwork.
<br><br>

Scrape financial information about the stock market
- **Scrape financial data from multiple sources**. You will get a couple of scripts that do all the hard work for you and scrape information about any stock ticker you provide.
<br><br>

Turn any transactions statement into an aggregated portfolio view

- **BONUS** if you have a portfolio! Transform your transactions statement into a dataset with daily snapshots of your portfolio and position sizes. I developed my own script to do this, and **I have included it in the course too**!

<div class="alert alert-block alert-info text-center">
    <H1> How to make the most of this course </H1>
</div>

### The course is divided in 3 sections

- We start with **Phase I on this document**, the data cleaning and transformation process. I'll keep it simple and all the scripts are available so we can focus on the important parts. You'll get a couple of tools I built for myself to scrape financial information about stocks. And I got a lot of messages about this function in one of the articles I wrote, so I'll give you my own script that **transforms a simple transactions list into a daily portfolio snapshot.**


- **Phase II** is all about **Plotly**, and we'll code together a nice **Cheat Sheet** while I show you the cool stuff Plotly can do. I'll use both Plotly Express and Plotly Graph Objects to add some flavor to your charts. Once you see it in action you will think twice before importing matplotlib again!


- **Phase III**, we explore **Dash** and learn how to setup a nice layout and theme using **Bootstrap**. I'll show you the basic elements and how to launch a simple app on your browser. Then we'll dive into the dashboard code and **we will create each page individually with the charts we already created in Phase II**.



<div class="alert alert-block alert-warning">
    <p> All the notebooks are inside the folder "jupyter notebooks" which is on the main folder of the project</p>
</div>

<div class="alert alert-block alert-info text-center">
    <H1> Requirements and setup </H1>
</div>

- If you are new to Python I recommend installing the Anaconda distribution (anaconda.com), which already has Jupyter Notebooks included.


- If you already know Python, you should be able to follow along just fine. We'll use a few packages that can easily be installed with "pip install" but other than that it will be straight forward.


- Again, you **don't need to have a portfolio to take advantage of this course**.

<div class="alert alert-block alert-info text-center">
    <H1> PHASE I </H1>
</div>

## From transactions list to transactions final

- Using the transactions from your brokers to create a cumulative view on your portfolio
- Expanding the table with Buy and Sell orders
- Save the dataframe

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

In [2]:
def expand_buyselldf(df):
    buysell_df = df.copy()
    buysell_df['transact_val'] = round(buysell_df['quantity'] * buysell_df['price'], 2)

    # Getting the previous row for the ticker
    prev_row = []
    for x, tick in enumerate(buysell_df['ticker']):
        if x == 0:
            prev_row.append(pd.NA)

        else:
            row_tick = buysell_df['ticker'][:x]
            last_occ = row_tick.where(row_tick == tick).last_valid_index()

            if last_occ is not None:
                prev_row.append(last_occ)
            else:
                prev_row.append(pd.NA)

    buysell_df['last_occurrence'] = prev_row
    buysell_df['last_occurrence'] = buysell_df['last_occurrence'].astype('Int64')

    # Getting the cashflow column
    cash_flow = []
    for x, ref in enumerate(buysell_df['type']):
        if ref == 'Buy':
            cash_flow.append(buysell_df['quantity'].iloc[x] * buysell_df['price'].iloc[x] * (-1))
        else:
            cash_flow.append(buysell_df['quantity'].iloc[x] * buysell_df['price'].iloc[x])

    buysell_df['cashflow'] = cash_flow
    buysell_df['cashflow'] = buysell_df['cashflow'].round(2)

    # Getting the previous units and the cumulative units for each row
    buysell_df['prev_units'] = 0.0
    buysell_df['cml_units'] = 0.0
    for x, ref in enumerate(buysell_df['last_occurrence']):
        if ref is pd.NA:
            buysell_df.iat[x,9] = 0
            if buysell_df['type'].iloc[x] == 'Buy':
                buysell_df.iat[x,10] = buysell_df['quantity'].iloc[x]

        else:
            buysell_df.iat[x,9] = buysell_df['cml_units'].iloc[ref]
            if buysell_df['type'].iloc[x] == 'Buy':
                buysell_df.iat[x,10] = round(buysell_df['cml_units'].iloc[ref] + buysell_df['quantity'].iloc[x], 4)
            else:
                buysell_df.iat[x,10] = round(buysell_df['cml_units'].iloc[ref] - buysell_df['quantity'].iloc[x], 4)

    # Getting the previous cost, cumulative cost, transtype cost and unit cost for each row
    buysell_df['prev_cost'] = 0.0 # 11
    buysell_df['cml_cost'] = 0.0 # 12
    buysell_df['cost_transact'] = 0.0 # 13
    buysell_df['cost_unit'] = 0.0 # 14

    for x, ref in enumerate(buysell_df['last_occurrence']):
        if ref is pd.NA:
            buysell_df.iat[x,11] = 0
            buysell_df.iat[x,13] = np.nan
            buysell_df.iat[x,14] = np.nan
            if buysell_df['type'].iloc[x] == 'Buy':
                buysell_df.iat[x,12] = buysell_df['transact_val'].iloc[x]
                buysell_df.iat[x,13] = np.nan
                buysell_df.iat[x,14] = np.nan
            # there should be no SELL on the first row!

        else: # in case last occurrence is not nan
            buysell_df.iat[x,11] = buysell_df['cml_cost'].iloc[ref]
            if buysell_df['type'].iloc[x] == 'Buy':
                buysell_df.iat[x,12] = round(buysell_df['cml_cost'].iloc[ref] + buysell_df['transact_val'].iloc[x], 4)
                buysell_df.iat[x,13] = np.nan
                buysell_df.iat[x,14] = np.nan
            else: # in case SELL
                buysell_df.iat[x,13] = round((buysell_df['quantity'].iloc[x]) / (buysell_df['cml_units'].iloc[ref]) * (buysell_df['cml_cost'].iloc[ref]), 4)
                buysell_df.iat[x,12] = round(buysell_df['cml_cost'].iloc[ref] - buysell_df['cost_transact'].iloc[x], 4)
                buysell_df.iat[x,14] = round(buysell_df['cml_cost'].iloc[ref] / buysell_df['cml_units'].iloc[ref], 4)

    # Getting the realized Gain/Loss and yield %
    buysell_df['gain_loss'] = 0.0
    buysell_df['yield'] = 0.0
    for x, ref in enumerate(buysell_df['type']):
        if ref == 'Sell':
            buysell_df.iat[x,15] = round(buysell_df['transact_val'].iloc[x] - buysell_df['cost_transact'].iloc[x], 4)
            buysell_df.iat[x,16] = round(buysell_df.iat[x,15] / buysell_df['cost_transact'].iloc[x], 4)
    return buysell_df.fillna(0)

def clean_header(df):
    df.columns = df.columns.str.strip().str.lower().str.replace('.', '', regex=False).str.replace('(', \
                '', regex=False).str.replace(')', '', regex=False).str.replace(' ', '_', regex=False).str.replace('_/_', '/', regex=False)
    
def get_now():
    now = datetime.now().strftime('%Y-%m-%d_%Hh%Mm')
    return now

def datetime_maker(df, datecol):
    df[datecol] = pd.to_datetime(df[datecol])

<div class="alert alert-block alert-info">
<b>Tip:</b> <br>If you have multiple brokers, you might want to add a script to merge them and format them together.
    <br>
    <br>You can also pick what stocks you want to track separately (for instance, one dashboard for Risky investments and another for Dividend stocks).
</div>

In [3]:
broker1_raw = pd.read_excel("../inputs/broker1/broker1.xlsx", engine='openpyxl')
broker1_raw.sort_index(inplace=True)
clean_header(broker1_raw)
datetime_maker(broker1_raw, 'time')
broker1_raw['no_of_shares'] = broker1_raw['no_of_shares'].round(4)
broker1_raw['action'].mask(broker1_raw['action'].str.contains('uy'), 'Buy', inplace=True)
broker1_raw['action'].mask(broker1_raw['action'].str.contains('ell'), 'Sell', inplace=True)
buysell_filter = (broker1_raw['action'].str.contains('Buy') | broker1_raw['action'].str.contains('Sell'))
cols_brok1 = ['time', 'action', 'ticker', 'no_of_shares', 'price/share', 'withholding_tax']
broker1_buysell = broker1_raw[buysell_filter][cols_brok1]
broker1_buysell.reset_index(inplace=True, drop=True)
cols_buysell = ['date', 'type', 'ticker', 'quantity', 'price', 'fees']
broker1_buysell.columns = cols_buysell
broker1_buysell['date'] = broker1_buysell['date'].dt.normalize()

In [4]:
broker2_raw = pd.read_csv("../inputs/broker2/broker2.csv", sep=';')
broker2_raw = broker2_raw[['Date', 'Type', 'Ticker', 'Quantity', 'Price per share', 'FX Rate']]
broker2_raw.columns = cols_buysell
broker2_raw.date = pd.to_datetime(broker2_raw.date, format="%d/%m/%Y %H:%M")
broker2_raw.type = broker2_raw.type.str.capitalize()
broker2_buysell = broker2_raw[broker2_raw.type.str.lower().str.contains('buy|sell')]
broker2_buysell.sort_values(by='date')
broker2_buysell.reset_index(inplace=True, drop=True)

In [5]:
merged_transactions = broker2_buysell.append(broker1_buysell)
merged_transactions.reset_index(inplace=True, drop=True)
merged_transactions.fillna(0, inplace=True)
merged_transactions.date = merged_transactions.date.dt.date
merged_transactions = merged_transactions.sort_values(by='date').reset_index(drop=True)

In [6]:
merged_transactions

,date,type,ticker,quantity,price,fees
0,2020-01-08,Buy,MMM,12.00,180.95,1.111550
1,2020-01-08,Buy,TSLA,1.26,95.40,1.111902
2,2020-01-08,Buy,NFLX,3.60,336.50,1.111803
3,2020-01-09,Buy,AAPL,24.00,77.25,1.110741
4,2020-02-03,Buy,MSFT,8.40,174.23,1.105987
...,...,...,...,...,...,...
99,2021-10-05,Buy,MO,20.00,46.44,0.000000
100,2021-10-13,Buy,RIOT,30.00,25.86,0.000000
101,2021-10-25,Buy,STOR,15.00,34.55,0.000000
102,2021-10-26,Sell,TSLA,3.00,1026.00,1.165200


<div class="alert alert-block alert-success">
<b>Run it!</b><br>Once you have the transactions ready, just call the function to expand the dataframe!
    <br><br>For the Excel die-hards, you can also use this table to do some quick exploring on a spreadsheet.
</div>

In [7]:
final = expand_buyselldf(merged_transactions).sort_values(by='date')

In [8]:
final.tail(10)

,date,type,ticker,quantity,price,fees,transact_val,last_occurrence,cashflow,prev_units,cml_units,prev_cost,cml_cost,cost_transact,cost_unit,gain_loss,yield
94,2021-03-05,Sell,NIO,20.0,37.15,0.0300,743.00,83,743.00,50.00,30.00,1411.30,846.7800,564.5200,28.2260,178.4800,0.3162
95,2021-05-06,Sell,MRNA,7.5,159.94,0.0900,1199.55,51,1199.55,12.50,5.00,839.50,335.8000,503.7000,67.1600,695.8500,1.3815
96,2021-05-17,Buy,PLUG,60.0,24.25,0.0000,1455.00,85,-1455.00,40.00,100.00,220.40,1675.4000,0.0000,0.0000,0.0000,0.0000
97,2021-07-30,Buy,AMZN,0.3,3358.69,0.0000,1007.61,33,-1007.61,1.40,1.70,3352.50,4360.1100,0.0000,0.0000,0.0000,0.0000
98,2021-08-04,Buy,RIOT,25.0,32.25,0.0000,806.25,0,-806.25,0.00,25.00,0.00,806.2500,0.0000,0.0000,0.0000,0.0000
99,2021-10-05,Buy,MO,20.0,46.44,0.0000,928.80,65,-928.80,30.00,50.00,1184.80,2113.6000,0.0000,0.0000,0.0000,0.0000
100,2021-10-13,Buy,RIOT,30.0,25.86,0.0000,775.80,98,-775.80,25.00,55.00,806.25,1582.0500,0.0000,0.0000,0.0000,0.0000
101,2021-10-25,Buy,STOR,15.0,34.55,0.0000,518.25,37,-518.25,32.00,47.00,734.40,1252.6500,0.0000,0.0000,0.0000,0.0000
102,2021-10-26,Sell,TSLA,3.0,1026.00,1.1652,3078.00,74,3078.00,11.46,8.46,2924.49,2158.9167,765.5733,255.1911,2312.4267,3.0205
103,2021-11-09,Sell,PLUG,45.0,41.17,0.0000,1852.65,96,1852.65,100.00,55.00,1675.40,921.4700,753.9300,16.7540,1098.7200,1.4573


In [9]:
final['cml_cashflow'] = final['cashflow'].cumsum()*-1

In [10]:
final['avg_price'] = final['cml_cost']/final['cml_units']

In [11]:
final.tail(10)

,date,type,ticker,quantity,price,fees,transact_val,last_occurrence,cashflow,prev_units,cml_units,prev_cost,cml_cost,cost_transact,cost_unit,gain_loss,yield,cml_cashflow,avg_price
94,2021-03-05,Sell,NIO,20.0,37.15,0.0300,743.00,83,743.00,50.00,30.00,1411.30,846.7800,564.5200,28.2260,178.4800,0.3162,53465.80,28.226000
95,2021-05-06,Sell,MRNA,7.5,159.94,0.0900,1199.55,51,1199.55,12.50,5.00,839.50,335.8000,503.7000,67.1600,695.8500,1.3815,52266.25,67.160000
96,2021-05-17,Buy,PLUG,60.0,24.25,0.0000,1455.00,85,-1455.00,40.00,100.00,220.40,1675.4000,0.0000,0.0000,0.0000,0.0000,53721.25,16.754000
97,2021-07-30,Buy,AMZN,0.3,3358.69,0.0000,1007.61,33,-1007.61,1.40,1.70,3352.50,4360.1100,0.0000,0.0000,0.0000,0.0000,54728.86,2564.770588
98,2021-08-04,Buy,RIOT,25.0,32.25,0.0000,806.25,0,-806.25,0.00,25.00,0.00,806.2500,0.0000,0.0000,0.0000,0.0000,55535.11,32.250000
99,2021-10-05,Buy,MO,20.0,46.44,0.0000,928.80,65,-928.80,30.00,50.00,1184.80,2113.6000,0.0000,0.0000,0.0000,0.0000,56463.91,42.272000
100,2021-10-13,Buy,RIOT,30.0,25.86,0.0000,775.80,98,-775.80,25.00,55.00,806.25,1582.0500,0.0000,0.0000,0.0000,0.0000,57239.71,28.764545
101,2021-10-25,Buy,STOR,15.0,34.55,0.0000,518.25,37,-518.25,32.00,47.00,734.40,1252.6500,0.0000,0.0000,0.0000,0.0000,57757.96,26.652128
102,2021-10-26,Sell,TSLA,3.0,1026.00,1.1652,3078.00,74,3078.00,11.46,8.46,2924.49,2158.9167,765.5733,255.1911,2312.4267,3.0205,54679.96,255.191099
103,2021-11-09,Sell,PLUG,45.0,41.17,0.0000,1852.65,96,1852.65,100.00,55.00,1675.40,921.4700,753.9300,16.7540,1098.7200,1.4573,52827.31,16.754000


<div class="alert alert-block alert-success">
<b>Ready to explore!</b><br>Save the file in the right folder so we can access it for Part 2.
</div>

In [16]:
final.to_excel('../outputs/transactions_all/transactions_finaldf_{}.xlsx'.format(get_now()), index=False)

In [12]:
# final.groupby(['ticker']).last().to_excel('../outputs/transactions_all/finaldf_lastpositions_{}.xlsx'.format(get_now()))